# Weryfikacja podziału dokumentów na fragmenty

Chunki utworzone skryptem `just chunk`
Sprawdzenie czy każdy chunk mieści się w oknie kontekstowym modeli

Wygląda, że problematyczne mogą być fragmenty tekstu z równaniami w latexu - dużo tokenów

Przy `chunk_size=700`, `chunk_overlap=20` wszystkie chunki mieszczą się w limicie 512 tokenów, ale średnia liczba tokenów to tylko ok 130.
Potencjalnie można by zwiększyć rozmiar chunków i pogodzić się z utratą informacji w niektórych fragmentach albo rozważyć inną strategię podziału

In [16]:
from pathlib import Path

import json

chunked_path = Path("../data/processed/chunked_notes/chunked_notes.json")
chunks = json.loads(chunked_path.read_text())
chunks[0]

{'id': 'ee6012c1-ec0a-40e9-8f30-435c9491a741',
 'content': '# Wprowadzenie do interfejsów i elementów aplikacji bazodanowych (2023-05-25)\n\n\n## Strategie współpracy z bazą danych\n* Osadzanie zapytań w uniwersalnym języku programowania\n  * Tekst zapytania stanowi część kodu źródłowego programu\n  * Zapytanie weryfikowane syntaktycznie w trakcie kompilacji\n  * Czytelny kod\n  * Zapytanie jest statyczne, nie można zmienić w trakcie wykonywania\n* Biblioteki typu JDBC, ODBC\n  * Możliwość generowanie zapytań w runtime\n  * brak weryfikacji podczas kompilacji\n* Natywne języki programowania baz danych (np. PL/SQL)\n  * wyzwalacze\n  * kursory\n    * jawne\n    * niejawne',
 'filename': '10-metody-dostepu-do-bazy-danych',
 'course': 'bazy-danych-1',
 'title': 'Wprowadzenie do interfejsów i elementów aplikacji bazodanowych (2023-05-25)',
 'number': 0}

In [3]:
from sentence_transformers import SentenceTransformer

bi_encoder_model = SentenceTransformer("sdadas/mmlw-retrieval-roberta-base")

In [4]:
bi_encoder_model.max_seq_length

512

In [17]:
n_tokens = []

for chunk in chunks:
    chunk_text = chunk["content"]
    chunk_tokens = bi_encoder_model.tokenizer.tokenize(chunk_text)
    n_chunk_tokens = len(chunk_tokens)
    n_tokens.append(n_chunk_tokens)
    if n_chunk_tokens > bi_encoder_model.max_seq_length:
        print(
            f"Chunk {chunk['id']} przekracza limit tokenów: {n_chunk_tokens} > {bi_encoder_model.max_seq_length}"
        )

print("Średnia liczba tokenów na chunk:", sum(n_tokens) / len(n_tokens))
print("Maksymalna liczba tokenów na chunk:", max(n_tokens))
print("Minimalna liczba tokenów na chunk:", min(n_tokens))

Średnia liczba tokenów na chunk: 129.76765585461237
Maksymalna liczba tokenów na chunk: 455
Minimalna liczba tokenów na chunk: 3
